
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 2 - Minimax Algorithm<br>
<font color=696880 size=4>
Sepehr Harfi


____

# Personal Data

In [18]:
Student_number = "400104801"
Name = "Mehran"
Last_name = "Bakhtiari"

# Rules

<font color=red>
Please run all the cells.

Please refrain from making any changes to the existing files, such as `board.py`, `game.py`, etc. 
</font>

#### **Note:** It is strongly recommended to execute this notebook on your local device instead of Google Colab due to limitations with graphics. However, if you still prefer using Google Colab, you can refer to this [link](https://vishnudsharma.medium.com/visualizing-tkinter-and-pygame-in-colab-272c5a245f8c) for assistance in using graphics with Colab.

# Libraries & Imports

**Dont change the below cell.**

In [1]:
import os

curr_working_directory = os.getcwd()
src_directory_path = os.path.join(curr_working_directory, "src")

if os.path.exists(src_directory_path):
    os.chdir(src_directory_path)

In [2]:
import game
from player import Player
from random_player import RandomPlayer
from random_greedy_player import RandomGreedyPlayer
import time


# Breakthrough Game: Minimax and AlphaBeta Players (100 Points)

In this notebook, we will implement and compare two AI strategies, Minimax and AlphaBeta, for playing a simple version of the Breakthrough game. We aim to assess the performance of these strategies in terms of their execution time and win probability in matches against a Greedy player and against each other with varying depths.


# Breakthrough Game Rules

The Breakthrough game is a two-player strategy board game played on an 6x6 grid. The objective of the game is to move your pieces from one end of the board to the other before your opponent does.

## Game Setup

- The board consists of a 6x6 grid with alternating dark and light squares.
- Each player starts with 12 pieces placed on their respective sides of the board.
- The pieces are initially arranged in two rows, with each row containing 6 pieces.

## Piece Movement

- Each player can only move their own pieces.
- Pieces can move diagonally or straight forward one square at a time.
- Pieces cannot move backward or sideways.
- Pieces can capture their opponent's pieces by moving diagonally forward and landing on a square occupied by an opponent's piece.
- Captured pieces are removed from the board.


## Game End

- If a player successfully moves one of their pieces to the opposite end of the board, they win the game.

## Additional Rules

- Players can only capture the opponent's piece if its in their diagonal forward squares and they cannot have two or more pieces of their color in the same square.
- Players take turns moving their pieces.
- Players must make a move on their turn, and they cannot skip their turn.
- If a player has multiple legal moves, they can choose which piece to move.


## Demo of the game with graphics

Initially, using the cell below, you can see a demo gameplay of two Random Greedy Players to gain insights into the game. Additionally, you can explore the gameplay of other agents such as "Player" which plays in a complete greedy way and also "RandomPlayer", to further enhance your understanding of the game.

**Note:** You can use the cell below anywhere you want to see the game with the graphics. For this purpose, you should only be careful to pass the right class of players to the `game.Game()`. Also, If you want to see the performance of your Minimax or AlphaBeta agents, you may need to add their classes as a python file to the `src` folder and import their classes in the cell below. 

In [3]:
%%python3 
# Change the above line to %%python if youre using Windows OS
import game
from player import Player
from random_greedy_player import RandomGreedyPlayer
from random_player import RandomPlayer


random_greedy_game = game.Game(RandomGreedyPlayer, RandomGreedyPlayer)
print(random_greedy_game.play(True))

{0: -175.0, 1: 227.0}



## Minimax Agent (30 Points)


To gain insights into the game, start by reading the `board.py`, `game.py`, and `player.py` files to understand the game implementation. Then, implement a minimax agent. Recall that the minimax algorithm evaluates game states and selects optimal moves. Compare the performance of the random greedy and minimax agents to gain insights.

**Note:** To implement the Minimax agent, you should only modify the `MinimaxPlayer` class in the cell below. You can either use the Board class `get_scores` function or define your own score funcion here and use it for the evaluation of a board state.

**Note:** Your minimax implementation should have a depth instance which determines the level to which the algorithm should be executed for each move. It controls the extent of the search tree explored by the algorithm.

**Note:** Feel free to add cells if you need to, but don't erase the existing cells. 

### Implementation (25 Points)

In [17]:
class MinimaxPlayer(Player):
    def __init__(self, player_number, board, depth=3):
        super().__init__(player_number, board)
        self.depth = depth

    def get_next_move(self):
        best_move, _ = self.minimax(self.depth, self.player_number, True)
        return best_move

    def minimax(self, depth, player, maximizing_player):
        if depth == 0 or self.board.is_game_over(self.board.get_board_grid()) != -1:
            return None, self.evaluate_board()

        if maximizing_player:
            max_score = float('-inf')
            best_move = None
            for move in self.get_possible_moves(player):
                self.board.start_imagination()
                self.board.place_piece_imaginary(move, player)
                _, score = self.minimax(depth - 1, self.opponent_number, False)
                if score > max_score:
                    max_score = score
                    best_move = move
            return best_move, max_score
        else:
            min_score = float('inf')
            best_move = None
            for move in self.get_possible_moves(player):
                self.board.start_imagination()
                self.board.place_piece_imaginary(move, player)
                _, score = self.minimax(depth - 1, self.opponent_number, True)
                if score < min_score:
                    min_score = score
                    best_move = move
            return best_move, min_score

    def evaluate_board(self):
        scores = self.board.get_scores(self.board.get_board_grid())
        return scores[self.player_number] - scores[self.opponent_number]

    def get_possible_moves(self, player):
        moves = []
        range_n = (0, self.board.get_n()) if player == self.player_number else (self.board.get_n() - 1, -1)
        step = 1 if player == self.player_number else -1
        for i in range(range_n[0], range_n[1], step):
            for j in range(self.board.get_n()):
                if self.board.get_board_grid()[i][j] == player:
                    for direction in self.board.get_possible_directions(player):
                        move = ((i, j), (i + direction[0], j + direction[1]))
                        if self.board.is_move_valid(self.board.get_board_grid(), move, player):
                            moves.append(move)
        return moves

### Time Analysis (5 Points)

Now play the game 5 times between two random players and calculate the average execution time. Also do this for the game of a random player and a minimax player. 

In [21]:
from minimax_player import MinimaxPlayer
from game import Game

def calculate_average_execution_time(player1_class, player2_class, num_games):
    total_time = 0
    for _ in range(num_games):
        start_time = time.time()
        game = Game(player1_class, player2_class)
        game.play()
        end_time = time.time()
        total_time += end_time - start_time
    return total_time / num_games

num_games = 5

mean_time_random = calculate_average_execution_time(RandomPlayer, RandomPlayer, num_games)

minimax_player = MinimaxPlayer

mean_time_minimax = calculate_average_execution_time(RandomPlayer, minimax_player, num_games)

print("Random Players Average Execution Time:", mean_time_random, "seconds")
print("Minimax Player and Random Player Average Execution Time:", mean_time_minimax, "seconds")


Random Players Average Execution Time: 0.004776763916015625 seconds
Minimax Player and Random Player Average Execution Time: 2.704791212081909 seconds


## Alphabeta Agent (30 Points)


As we can see from the above code, the Minimax algorithm takes much longer time to execute. To improve the execution time, we can implement the AlphaBeta pruning algorithm. In the next cell, we will be implementing the AlphaBeta player.


### Implementation (25 Points)

In [3]:
class AlphaBetaPlayer(Player):
    def __init__(self, player_number, board, depth=3):
        super().__init__(player_number, board)
        self.depth = depth
        
    def get_scores(self, board):
        scores = {0: 0, 1: 0}
        for row in range(6):
            for col in range(6):
                if board[row][col] == 0:
                    scores[0] += 1 + (2.0 ** row) * row
                elif board[row][col] == 1:
                    scores[1] += 1 + (2.0 ** (6 - row - 1)) * (6 - row - 1)
        scores[0] += self.board.winning_score(board, 0)
        scores[1] += self.board.winning_score(board, 1)
        return scores
                
    def get_next_move(self):
        max_score = -float('inf')
        best_move = None
        for move, i, j, dx, dy in self.get_possible_moves(self.player_number):
            self.board.start_imagination()
            self.board.place_piece_imaginary(move, self.player_number)
            score = self.alphabeta(depth=1, alpha=-float('inf'), beta=float('inf'))
            if score > max_score:
                max_score = score
                best_move = move
        return best_move
        
    def alphabeta(self, depth, alpha, beta):
        board = self.board.get_imaginary_board()
        if self.board.is_game_over(board) != -1 or depth == self.depth:
            scores = self.get_scores(board)
            return scores[self.player_number] - scores[self.opponent_number]
        
        if depth % 2 == 0:
            max_score = -float('inf')
            for move, i, j, dx, dy in self.get_possible_moves(self.player_number):
                previous_destination_value = board[i][j]
                self.board.place_piece_imaginary(move, self.player_number)
                score = self.alphabeta(depth + 1, alpha, beta)
                max_score = max(max_score, score)
                board[i][j] = previous_destination_value
                board[i + dx][j + dy] = 0
                if score >= beta:
                    return max_score
                alpha = max(alpha, score)
            return max_score
        else:
            min_score = float('inf')
            for move, i, j, dx, dy in self.get_possible_moves(self.opponent_number):
                previous_destination_value = board[i][j]
                self.board.place_piece_imaginary(move, self.opponent_number)
                score = self.alphabeta(depth + 1, alpha, beta)
                min_score = min(min_score, score)
                board[i][j] = previous_destination_value
                board[i + dx][j + dy] = 1
                if score <= alpha:
                    return min_score
                beta = min(beta, score)
            return min_score
        
    def get_possible_moves(self, player_number):
        moves = []
        range_n = (0, self.board.get_n()) if player_number == self.player_number else (self.board.get_n() - 1, -1)
        step = 1 if player_number == self.player_number else -1

        board = self.board.get_imaginary_board()
        if board is None:
            return moves
        
        for i in range(range_n[0], range_n[1], step):
            for j in range(range_n[0], range_n[1], step):
                if board[i][j] == player_number:
                    for direction in self.board.get_possible_directions(player_number):
                        move = (i, j), (i + direction[0], j + direction[1])
                        if self.board.is_move_valid(board, move, player_number):
                            moves.append((move, i, j, direction[0], direction[1]))
        return moves

### Time Analysis (5 Points)

Now play the game 5 times between the random player and an alphabeta player and calculate the average execution time. 

In [13]:
from alphabeta_player import AlphaBetaPlayer
from game import Game

def calculate_average_execution_time(player1_class, player2_class, num_games):
    total_time = 0
    for _ in range(num_games):
        start_time = time.time()
        game = Game(player1_class, player2_class)
        game.play()
        end_time = time.time()
        total_time += end_time - start_time
    return total_time / num_games

num_games = 5

random_player = RandomPlayer
alphabeta_player = AlphaBetaPlayer

mean_time_alphabeta = calculate_average_execution_time(random_player, alphabeta_player, num_games)

print("Alphabeta Player and Random Player Average Execution Time:", mean_time_alphabeta, "seconds")

Alphabeta Player and Random Player Average Execution Time: 0.514774751663208 seconds


##### **Note:** The alphabeta agent should be approximately 5X-10X faster than the minimax player.


## Win Probability Analysis (35 Points)

In this section, We will simulate 50 games for your AlphaBeta player against other players and analyze their win rates. Additionally, we will have AlphaBeta players with different depths play against each other. 

Assume you are always the second player and the black player will always start first. 

If the agent is implemented correctly, with a depth of two or more it should win all the mentioned agents with a win rate > 0.7.

**Note:** You can use the `bulk_play` function from `game.py` for this purpose.


In [5]:
import pandas as pd

depths = [1, 2, 3]  # List of different second_player_depth values

In [6]:
def compute_results_different_depths(first_player_class, depths, first_player_depth=None, n=50):
    results = []
    sample_game = game.Game(first_player_class, AlphaBetaPlayer)

    for depth in depths:
        if first_player_depth is not None:
            result = sample_game.bulk_play(n, first_player_depth=first_player_depth, second_player_depth=depth)
        else:
            result = sample_game.bulk_play(n, second_player_depth=depth)
        win_rate = result['white'] / n  # Calculate win rate
        results.append({'Depth': depth, 'Win Rate': win_rate})

    df = pd.DataFrame(results)
    return df

### Random Player

In [7]:
results = compute_results_different_depths(RandomPlayer, depths)
print(results)

   Depth  Win Rate
0      1       1.0
1      2       1.0
2      3       1.0


### Random Greedy Player

In [8]:
results = compute_results_different_depths(RandomGreedyPlayer, depths)
print(results)

   Depth  Win Rate
0      1       1.0
1      2       1.0
2      3       1.0


### Greedy Player

In [9]:
results = compute_results_different_depths(Player, depths)
print(results)

   Depth  Win Rate
0      1       1.0
1      2       1.0
2      3       1.0


### Alphabeta Player with depth 2

**Note:** In this part each game may take up to ~45 seconds.

In [13]:
results = compute_results_different_depths(AlphaBetaPlayer, [2, 3, 4], first_player_depth=2, n=30)
print(results)

   Depth  Win Rate
0      2       0.0
1      3       1.0
2      4       1.0



## Conclusion (5 Points)

#### Based on the results of the AlphaBeta player with other players, what is your conclusion?


**Answer:**
Based on the results obtained from simulating games between the AlphaBeta player and other player types, we can draw several conclusions:

Random Player: The AlphaBeta player consistently outperforms the Random Player across all depths tested. This is expected since the Random Player makes moves without considering any strategy, while the AlphaBeta player employs a sophisticated search algorithm to select optimal moves.

Random Greedy Player: Similar to the Random Player, the AlphaBeta player also dominates the Random Greedy Player across all depths tested. Although the Random Greedy Player has some heuristic strategy (greediness), it's not sufficient to overcome the strategic depth of the AlphaBeta player.

Greedy Player: The Greedy Player performs better than the Random Player and the Random Greedy Player. However, it still falls short compared to the AlphaBeta player. The Greedy Player prioritizes immediate gains without considering long-term consequences, whereas the AlphaBeta player looks ahead to future moves to optimize its strategy.

AlphaBeta Player with depth 2 vs. depth 2, 3, 4: As expected, increasing the depth of the AlphaBeta player's search generally leads to better performance. However, there might be diminishing returns beyond a certain depth, as the computational complexity increases with depth. In this case, the win rate might not improve significantly beyond a depth of 3.

Overall, the results indicate that the AlphaBeta player, especially with deeper search depths, is a strong player capable of consistently outperforming players that lack strategic foresight, such as random or greedy players. 